Algorytm MRP:

Define DataFrame:

In [19]:
import pandas as pd
ilosc_tyg = 6

#calk_zapot_list = [0,0,0,28,0,30]
#calk_zapot_list = [40,0,0,0,0,0]
#calk_zapot_list = [0,0,0,112,0,120]

# blaty = {
#     "calk_zapot_list":[0,0,0,28,0,30],
#     "czas_realizacji":3,
#     "wielkosc_partii":40,
#     "na_stanie":22
# }
# plyta = {
#     "calk_zapot_list":[40,0,0,0,0,0],
#     "czas_realizacji":1,
#     "wielkosc_partii":50,
#     "na_stanie":10
# }
# nogi = {
#     "calk_zapot_list":[0,0,0,112,0,120],
#     "czas_realizacji":2,
#     "wielkosc_partii":120,
#     "na_stanie":40
# }
gryf = {
    "calk_zapot_list":[0,0,0,28,0,30],
    "czas_realizacji":3,
    "wielkosc_partii":40,
    "na_stanie":22
}
zelazo = {
    "calk_zapot_list":[40,0,0,0,0,0],
    "czas_realizacji":1,
    "wielkosc_partii":50,
    "na_stanie":10
}
pokrywa_gumowa = {
    "calk_zapot_list":[0,0,0,112,0,120],
    "czas_realizacji":2,
    "wielkosc_partii":120,
    "na_stanie":40
}
calk_zapot_list, czas_realizacji, wielkosc_partii, na_stanie  = pokrywa_gumowa.values() ### <--- tutaj zmieniaj przyklady :)
#calk_zapot_list = zad1["calk_zapot_list"]
empt = [0 for x in range(ilosc_tyg)]

#LABELS
calk_zap = "calkowite zapotrzebowanie"
plan_przyj = "planowane przyjecia"
przew_na_stn = "przewidywane na stanie"
zapo_nett = "zapotrzebowanie netto"
plan_zam = "planowane zamowienia"
plan_przyj_zam = "planowane przyjecie zamowien"
#

data = {
    calk_zap:calk_zapot_list,
    plan_przyj:empt,
    przew_na_stn:empt,
    zapo_nett:empt,
    plan_zam:empt,
    plan_przyj_zam:empt
}
mrp = pd.DataFrame(data=data, index=[x for x in range(1,ilosc_tyg+1)], dtype="int64")
mrp.transpose()

,1,2,3,4,5,6
calkowite zapotrzebowanie,0,0,0,112,0,120
planowane przyjecia,0,0,0,0,0,0
przewidywane na stanie,0,0,0,0,0,0
zapotrzebowanie netto,0,0,0,0,0,0
planowane zamowienia,0,0,0,0,0,0
planowane przyjecie zamowien,0,0,0,0,0,0


Define other constants: (czas realizacji, wielkosc partii, poziom BOM, na stanie, mnoznik czesci)

In [13]:
#czas_realizacji = 3  # zdef wyzej dla danego przykladu   
#wielkosc_partii = 40
poziom_BOM = 1
#na_stanie = 22      
mnoznik_czesci = 1
#calk_zapot_list, czas_realizacji, wielkosc_partii, na_stanie  = zad1.values()

calculate MRP:

In [20]:
for aktu_tydz in range(1, ilosc_tyg+1):
    if aktu_tydz == 1:
        wpns = na_stanie # wpns <=> wczesniejsze przewidywane na stanie
    else:
        wpns = mrp.loc[aktu_tydz-1,przew_na_stn]
    
    zn = wpns - mrp.loc[aktu_tydz,calk_zap]  # zn <=> zapotrzebowanie netto (z mozliwym minusem)
    if zn < 0:
        ppz = wielkosc_partii # ppz <=> planowane przyjecie zamowien
        while ppz < zn:
            ppz += wielkosc_partii # petla zebysmy dostali wystarczajaco produktu
            
        mrp.loc[aktu_tydz,zapo_nett] = abs(zn)
        if aktu_tydz - czas_realizacji >= 1:
            mrp.loc[aktu_tydz-czas_realizacji,plan_zam] = ppz
            mrp.loc[aktu_tydz,plan_przyj_zam] = ppz
        else:
            print("Nie wystarczy czasu!")
            mrp.loc[1, plan_zam] = ppz
            mrp.loc[1+czas_realizacji,plan_przyj_zam] = ppz
            
        mrp.loc[aktu_tydz,przew_na_stn] = mrp.loc[aktu_tydz,plan_przyj_zam] - abs(zn)
    else:   
        mrp.loc[aktu_tydz,przew_na_stn] = zn 
    
    

In [21]:
mrp.transpose()

,1,2,3,4,5,6
calkowite zapotrzebowanie,0,0,0,112,0,120
planowane przyjecia,0,0,0,0,0,0
przewidywane na stanie,40,40,40,48,48,48
zapotrzebowanie netto,0,0,0,72,0,72
planowane zamowienia,0,0,120,0,120,0
planowane przyjecie zamowien,0,0,0,120,0,120
